In [2]:
!pip install -q kaggle
import json

data = {"username":"tunkedsaro","key":"ff3d00dc680ff94caf6f8d4d9bcd3168"}
json_data = json.dumps(data)

# Write the JSON string to a file
file_path = "/content/kaggle.json"
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c forest-type-classification-spai

ref                                                             title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
shreyanshverma27/online-sales-dataset-popular-marketplace-data  Online Sales Dataset - Popular Marketplace Data      7KB  2024-05-25 23:55:26           2942         52  1.0              
devarajv88/walmart-sales-dataset                                e-Commerce (Walmart) Sales Dataset                   5MB  2024-05-30 12:09:55           1925         31  1.0              
ayushparwal2026/country-population-from-1960-to-2022            Country Population from 1960 to 2022                75KB  2024-05-31 08:58:26           1458         39  0.9411765        
mayankanand2701/apple-stock-price-dataset                       A

In [3]:
!unzip /content/forest-type-classification-spai.zip

Archive:  /content/forest-type-classification-spai.zip
  inflating: metaData.csv            
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


<hr>

In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split


In [5]:
df = pd.read_csv("/content/train.csv")
# train_df = train_df[0:20]
df

,id,b1,b11,b12,b2,b3,b4,b5,b6,b7,b8,b8_a,b9,nforest_type
0,2002,293,1927,1038,278,475,453,987,1773,2184,1900,2343,3039,MDF
1,3212,197,1598,697,201,347,228,682,1982,2449,2254,2685,2690,DDF
2,13312,929,1975,1031,982,1020,856,1220,2051,2421,2392,2671,2683,MDF
3,17020,132,1560,689,189,408,175,609,2117,2907,3024,3005,2955,MDF
4,5967,241,1944,1131,362,538,487,918,1549,1844,1702,2077,2043,MDF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13048,9185,374,1940,1054,382,565,498,977,1678,1929,2109,2291,2100,DDF
13049,13977,1983,3602,2720,1622,1782,1766,2314,3488,3900,3924,4097,6053,DDF
13050,755,940,2007,1148,975,1080,968,1252,1780,1983,1942,2247,2170,DDF
13051,1616,1174,2312,1190,1112,1126,889,1310,2511,3085,3050,3396,3380,MDF


In [6]:
label_encoder = LabelEncoder()
df['nforest_type'] = label_encoder.fit_transform(df['nforest_type'])
df

,id,b1,b11,b12,b2,b3,b4,b5,b6,b7,b8,b8_a,b9,nforest_type
0,2002,293,1927,1038,278,475,453,987,1773,2184,1900,2343,3039,2
1,3212,197,1598,697,201,347,228,682,1982,2449,2254,2685,2690,0
2,13312,929,1975,1031,982,1020,856,1220,2051,2421,2392,2671,2683,2
3,17020,132,1560,689,189,408,175,609,2117,2907,3024,3005,2955,2
4,5967,241,1944,1131,362,538,487,918,1549,1844,1702,2077,2043,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13048,9185,374,1940,1054,382,565,498,977,1678,1929,2109,2291,2100,0
13049,13977,1983,3602,2720,1622,1782,1766,2314,3488,3900,3924,4097,6053,0
13050,755,940,2007,1148,975,1080,968,1252,1780,1983,1942,2247,2170,0
13051,1616,1174,2312,1190,1112,1126,889,1310,2511,3085,3050,3396,3380,2


In [7]:
train_df, val_df = train_test_split(df, test_size=0.3)

In [8]:
class ForestDataset(Dataset):
  def __init__(self,df):
    self.data = df.drop(['id','nforest_type'],axis=1).values
    self.labels = df['nforest_type'].values
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    return torch.tensor(self.data[idx],dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

In [9]:
# Create DataLoader for training
train_dataset = ForestDataset(train_df)
val_dataset = ForestDataset(val_df)
train_dataset[0]

(tensor([ 496., 2084., 1270.,  457.,  622.,  522.,  952., 1943., 2129., 1831.,
         2418., 2492.]),
 tensor(2))

In [10]:
val_dataset[0]

(tensor([2108., 3699., 2915., 2088., 2276., 2172., 2615., 3636., 3978., 3864.,
         4214., 5664.]),
 tensor(2))

In [11]:
# อันนี้สำหรับ เข้าโมเดลของ train กับ valid
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader   = DataLoader(val_dataset,batch_size=32)

In [12]:
import torch.nn as nn
import torch.nn.functional as F

In [13]:
# class NeuralNet(nn.Module):
#   def __init__(self):
#     super(NeuralNet, self).__init__()
#     self.fc1 = nn.Linear(12, 64)
#     self.fc2 = nn.Linear(64, 64)
#     self.fc3 = nn.Linear(64, len(label_encoder.classes_))
#   def forward(self,x):
#     x = F.relu(self.fc1(x))
#     x = F.relu(self.fc2(x))
#     x =self.fc3(x)
#     return x

# model = NeuralNet()

In [14]:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# for epoch in range(100):  # Or however many epochs you want
#     running_loss = 0.0
#     for i, data in enumerate(dataloader, 0):
#         inputs, labels = data
#         # print(inputs,labels)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#     print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader)}")

In [15]:
# class NeuralNet(nn.Module):
#   def __init__(self):
#     super(NeuralNet, self).__init__()
#     # self.fc1 = nn.Linear(12, 64)
#     # self.fc2 = nn.Linear(64, 64)
#     # self.fc3 = nn.Linear(64, len(label_encoder.classes_))

#     self.fcinput = nn.Linear(12,128)

#     self.fc1 = nn.Dropout(0.1)
#     self.fc2 = nn.Linear(128,128)
#     self.fc3 = nn.ReLU()

#     self.fcoutput = nn.Linear(128,len(label_encoder.classes_))
#     self.main_block = nn.Sequential(*[
#         self.fcinput,
#         self.fc1,
#         self.fc2,
#         self.fc3,
#         self.fcoutput
#     ])

#   def forward(self,x):
#     return self.main_block(x)

# model = NeuralNet()

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from tqdm import tqdm


In [17]:
# https://github.com/autogluon/autogluon/blob/9b44d85668b627893d6df4b0d7d3993875b55f8a/tabular/src/autogluon/tabular/models/tabular_nn/hyperparameters/parameters.py#L14
class NeuralNet(nn.Module):
  def __init__(self):
    super(NeuralNet, self).__init__()
    # self.fc1 = nn.Linear(12, 64)
    # self.fc2 = nn.Linear(64, 64)
    # self.fc3 = nn.Linear(64, len(label_encoder.classes_))

    layers = []
    layers.append(nn.Linear(12,128))

    for i in range(4):
        layers.append(nn.Dropout(0.1))
        layers.append(nn.Linear(128,128))
        layers.append(nn.ReLU())

    layers.append(nn.Linear(128,len(label_encoder.classes_)))
    self.main_block = nn.Sequential(*layers)

  def forward(self,x):
    return self.main_block(x)

model = NeuralNet()
model.to('cuda')       # If CPU comment

NeuralNet(
  (main_block): Sequential(
    (0): Linear(in_features=12, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.1, inplace=False)
    (5): Linear(in_features=128, out_features=128, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=128, out_features=128, bias=True)
    (9): ReLU()
    (10): Dropout(p=0.1, inplace=False)
    (11): Linear(in_features=128, out_features=128, bias=True)
    (12): ReLU()
    (13): Linear(in_features=128, out_features=3, bias=True)
  )
)

In [18]:
# # CPU version
# # Assuming 'model' and 'dataloader' are already defined
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.AdamW(model.parameters(), lr=0.001)

# # Define validation function
# def validate(model, dataloader):
#     model.eval()  # Set the model to evaluation mode
#     val_loss = 0.0
#     all_preds = []
#     all_labels = []
#     with torch.no_grad():
#         for data in dataloader:
#             inputs, labels = data
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             val_loss += loss.item()
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#     val_loss /= len(dataloader)
#     accuracy = accuracy_score(all_labels, all_preds)
#     return val_loss, accuracy
# best_val_accuracy = 0.0
# best_model_path = '/content/best_model/best_model3.pt'

# for epoch in tqdm(range(1000)):  # Or however many epochs you want
#     model.train()  # Set the model to training mode
#     running_loss = 0.0
#     for i, data in enumerate(train_dataloader, 0):
#         inputs, labels = data
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()

#     avg_training_loss = running_loss / len(train_dataloader)
#     val_loss, val_accuracy = validate(model, val_dataloader)  # Assuming 'val_dataloader' is your validation dataloader

#     if val_accuracy > best_val_accuracy:
#         best_val_accuracy = val_accuracy
#         torch.save(model.state_dict(), best_model_path)
#         print(f"New best model saved with validation accuracy: {best_val_accuracy:.4f}")

#     print(f"Epoch {epoch+1}, Training Loss: {avg_training_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f} AccDiff: {(val_accuracy - best_val_accuracy):.4f}")

  0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [23]:
# GPU version
# Define the model architecture
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        layers = []
        layers.append(nn.Linear(12, 128))
        for i in range(4):
            layers.append(nn.Dropout(0.1))
            layers.append(nn.Linear(128, 128))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(128, len(label_encoder.classes_)))
        self.main_block = nn.Sequential(*layers)

    def forward(self, x):
        return self.main_block(x)

# Initialize the model and move it to GPU
model = NeuralNet().to('cuda')

# Assuming 'model' and 'dataloader' are already defined
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Define validation function
def validate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            inputs, labels = inputs.to('cuda'), labels.to('cuda')  # Move inputs and labels to GPU
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    val_loss /= len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    return val_loss, accuracy

# Initialize variables for tracking the best model
best_val_accuracy = 0.0
best_model_path = '/content/best_model/best_model3.pt'

# Training loop
for epoch in range(100):  # Or however many epochs you want
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda')  # Move inputs and labels to GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_training_loss = running_loss / len(train_dataloader)
    val_loss, val_accuracy = validate(model, val_dataloader)  # Assuming 'val_dataloader' is your validation dataloader

    # Save the model if the validation accuracy is the best we've seen so far
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved with validation accuracy: {best_val_accuracy:.4f}")

    print(f"Epoch {epoch+1}, Training Loss: {avg_training_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, AccDiff: {(val_accuracy - best_val_accuracy):.4f}")


New best model saved with validation accuracy: 0.4671
Epoch 1, Training Loss: 1.5531, Validation Loss: 0.9598, Validation Accuracy: 0.4671, AccDiff: 0.0000
New best model saved with validation accuracy: 0.5138
Epoch 2, Training Loss: 1.0205, Validation Loss: 0.9618, Validation Accuracy: 0.5138, AccDiff: 0.0000
New best model saved with validation accuracy: 0.5156
Epoch 3, Training Loss: 0.9960, Validation Loss: 0.9511, Validation Accuracy: 0.5156, AccDiff: 0.0000
Epoch 4, Training Loss: 0.9792, Validation Loss: 0.9439, Validation Accuracy: 0.5140, AccDiff: -0.0015
Epoch 5, Training Loss: 0.9642, Validation Loss: 0.9370, Validation Accuracy: 0.5100, AccDiff: -0.0056
New best model saved with validation accuracy: 0.5490
Epoch 6, Training Loss: 0.9573, Validation Loss: 0.9232, Validation Accuracy: 0.5490, AccDiff: 0.0000
New best model saved with validation accuracy: 0.5577
Epoch 7, Training Loss: 0.9427, Validation Loss: 0.9199, Validation Accuracy: 0.5577, AccDiff: 0.0000
Epoch 8, Train

In [26]:
model.load_state_dict(torch.load("/content/best_model/best_model3.pt"))
val_loss, val_accuracy = validate(model, val_dataloader)  # Assuming 'val_dataloader' is your validation dataloader
val_loss, val_accuracy

(0.7855498911888619, 0.6327885597548519)

<hr>

In [24]:
import pandas as pd
import torch

In [25]:
test_df = pd.read_csv("/content/test.csv")
# test_df = test_df[0:10]
test_df

,id,b1,b11,b12,b2,b3,b4,b5,b6,b7,b8,b8_a,b9
0,13467,69,1425,693,312,524,376,847,1821,2356,2378,2611,2595
1,12719,242,1514,691,343,522,324,718,1730,2178,2472,2359,2582
2,1054,218,2354,1118,292,596,410,965,2586,3226,3371,3645,3149
3,13747,350,2013,1134,306,572,475,982,1754,1935,2275,2290,2345
4,9453,185,1450,712,293,440,384,673,1487,1965,2213,2200,2193
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,115,447,1686,811,425,661,441,958,2432,2891,2966,3126,3312
3996,10654,252,2694,1503,470,778,753,1294,2334,2656,2679,3212,2856
3997,5718,233,1486,618,249,409,260,699,2188,2831,3030,3086,3087
3998,13054,221,1840,774,245,441,231,703,2491,3453,3284,3762,3161


In [27]:
test_data = test_df.drop('id',axis=1).values
test_data

array([[  69, 1425,  693, ..., 2378, 2611, 2595],
       [ 242, 1514,  691, ..., 2472, 2359, 2582],
       [ 218, 2354, 1118, ..., 3371, 3645, 3149],
       ...,
       [ 233, 1486,  618, ..., 3030, 3086, 3087],
       [ 221, 1840,  774, ..., 3284, 3762, 3161],
       [   2, 1431,  629, ..., 2113, 2645, 2630]])

In [31]:
input_data = torch.tensor(test_data, dtype=torch.float32)
# input_data   # CPU
input_data = input_data.to('cuda')  # GPU

In [32]:
model.eval()
with torch.no_grad():
  outputs = model(input_data)

In [33]:
outputs

tensor([[-2.2110e+00,  9.5976e-01, -1.2438e+00],
        [-2.9986e-01, -1.1033e+00, -3.9331e-01],
        [-1.8433e+00, -3.6891e+00, -2.9188e-01],
        ...,
        [-2.0548e-03,  2.1016e-01, -5.6388e-01],
        [-1.7427e+00, -1.9994e+00, -5.8051e-01],
        [-3.9864e+00,  1.5622e+00, -1.7903e+00]], device='cuda:0')

In [34]:
# Get the predicted class
predicted_class = torch.argmax(outputs, dim=1)
predicted_class

tensor([1, 0, 2,  ..., 1, 2, 1], device='cuda:0')

In [38]:
# # CPU
# predicted_labels = label_encoder.inverse_transform(predicted_class.numpy().flatten())
# GPU
predicted_labels = label_encoder.inverse_transform(predicted_class.cpu().numpy().flatten())

print(predicted_labels)

['DEF' 'DDF' 'MDF' ... 'DEF' 'MDF' 'DEF']


In [39]:
submit = pd.read_csv("/content/sample_submission.csv")
submit

,id,nforest_type
0,13467,DEF
1,12719,DDF
2,1054,MDF
3,13747,NaN
4,9453,NaN
...,...,...
3995,115,NaN
3996,10654,NaN
3997,5718,NaN
3998,13054,NaN


In [40]:
# Insert predictions into the DataFrame
submit['nforest_type'] = predicted_labels
submit

,id,nforest_type
0,13467,DEF
1,12719,DDF
2,1054,MDF
3,13747,DDF
4,9453,DEF
...,...,...
3995,115,DDF
3996,10654,MDF
3997,5718,DEF
3998,13054,MDF


In [ ]:
submit.to_csv("v.2.1.4.1155.csv",index=False)

<hr>

In [ ]:

# 0:15 /
df['ATSAVI'] = (df['09NIRn'] - (0.33 * df['04Red']) - (0.5 * df['08NIR'] + df['04Red'] - (0.5 * 0.33)) + (1 + (0.33 ** 2)))
df['AFRI1600'] = (df['08NIR'] - 0.6616*df['08NIR'] + 0.6616*df['08NIR'])
df['AFRI2100'] = (df['08NIR'] - 0.521 * df['08NIR'] + 0.562 * df['08NIR'])
df['ARI'] = (df['12SWIR1'] - df['13SWIR2']) / (df['12SWIR1'] + df['13SWIR2'])
df['ARVI'] = (df['08NIR'] - df['04Red'] - (df['08NIR'] - df['02Blue']))/(df['08NIR'] + df['04Red'] - (df['08NIR'] - df['02Blue']))
df['ARVI2'] = -0.18+1.17*((df['08NIR']-df['04Red'])/(df['08NIR']+df['04Red']))
df['BWDRVI'] = (df['08NIR'] - df['02Blue']) / (df['08NIR'] + df['02Blue'])
df['BRI'] = df['12SWIR1'] - df['13SWIR2']
df['CCCI'] = (df['08NIR'] - df['07Veg7']) / (df['08NIR'] + df['07Veg7'] - df['04Red'] + df['04Red'])
df['CARI2'] = (((np.abs(df['08NIR'] - df['06Veg6']) + np.abs(df['08NIR'] + df['06Veg6'])) ** 0.5) * df['07Veg7'])
df['Chlgreen'] = (df['07Veg7'] - df['03Green']) * (-1)
df['CIgreen'] = df['08NIR'] - df['03Green']
df['CIrededge'] = df['09NIRn'] - 1
df['ChlRed-Edge'] = (df['08NIR'] - df['07Veg7']) * -1
df['CI'] = df['04Red'] - df['02Blue']
# 16:30
df['CRI700'] = (-1) * df['01Coastal'] + (-1) * df['03Green']
df['GDVI'] = df['08NIR'] - df['03Green']
df['EVI'] = 2.5 * df['08NIR'] - df['04Red'] * (df['08NIR'] + 6 * df['04Red'] - 7.5 * df['02Blue'] + 1)
df['EVI2'] = 2.4 * df['08NIR'] - df['04Red'] / (df['08NIR'] + df['04Red'] + 1)
df['EVI2'] = 2.5 * df['08NIR'] - df['04Red'] * df['08NIR'] + 2.4 *  df['04Red'] + 1
df['EPI'] = (df['05Veg5'] * df['02Blue'] * df['03Green'])
df['GEMI'] = (df['08NIR'] - (0.25 * df['08NIR'])) - (df['04Red'] + (0.125 * df['04Red']))
df['GVMI'] = (df['08NIR'] + 0.1) - (df['12SWIR1'] + 0.02) / (df['08NIR'] + 0.1) + (df['12SWIR1'] + 0.02)
df['GARI'] = (df['08NIR'] - (df['03Green'] - (df['02Blue'] - df['04Red']))) / (df['08NIR'] + (df['03Green'] + (df['02Blue'] - df['04Red'])))
df['GLI'] = 2 * df['03Green'] - df['04Red'] - df['02Blue'] / (2 * df['03Green'] + df['04Red'] + df['02Blue'])
df['GNDVI'] = df['08NIR'] - df['03Green'] / (df['08NIR'] + df['03Green'])
df['GOSAVI'] = df['08NIR'] - df['09NIRn'] + df['03Green'] + df['04Red']
df['GSAVI'] = df['08NIR'] - df['09NIRn'] + df['03Green'] + (0.5 * (1 + 0.5))
# 31:45
df['IVI'] = df['08NIR'] - df['04Red'] * 2
df['NDVI'] = (df['08NIR'] - df['04Red']) / (df['08NIR'] + df['04Red'])
df['IPVI'] = 1/(df['08NIR']/df['10WV'] + (df['04Red']/df['10WV'])**2) * (df['NDVI'] + 1)
df['IR700'] = 1/(df['07Veg7']+1e-8)
df['LCI'] = (df['01Coastal'] - df['06Veg6']) / (df['01Coastal'] + df['06Veg6'])
df['LogR'] = np.log(df['08NIR'] / df['04Red'])
df['MCARI/OSAVI'] = (
    (df['09NIRn'] - df['07Veg7'])
    + (-0.2 * (df['09NIRn'] - df['05Veg5']))
    * ((df['09NIRn'] - df['07Veg7'])
       * (1 + 0.16))
    / (df['08NIR'] - df['07Veg7'])
    + (df['08NIR'] + df['07Veg7']) + 0.16
)
df['mCRIRE'] = (-(df['03Green'] - df['06Veg6'])+df['08NIR'])
df['MVI'] = (df['12SWIR1'] + df['13SWIR2']) / 2
df['MCARI1'] = 1.2 * (2.5 * (df['08NIR'] - df['06Veg6']) - 1.3 * (df['08NIR'] - df['03Green']))
df['mNDVI'] = (df['08NIR'] - df['06Veg6']) / (df['08NIR'] + df['06Veg6'] - df['10WV'])
# 46:60
df['MSR670'] = ((df['08NIR'] - df['06Veg6']) / (df['08NIR'] + df['06Veg6']))**0.5
df['MSAVIhyper'] = (0.5)*((df['10WV']+1) - np.sqrt((df['10WV']+1)**2 - 8*(df['08NIR'] - df['05Veg5'])))
df['MTVI1'] = 1.2 * (1.2 * (df['08NIR'] - df['03Green']) - 2.5 * (df['06Veg6'] - df['03Green']))
df['MTVI2'] = (
    1.5 * (df['08NIR'] - df['05Veg5'])
    - 2.5 * (df['06Veg6'] - df['05Veg5'])
    * ((df['08NIR'] + 1) ** 2 - (df['06Veg6'] - 5 * df['06Veg6'] ** 0.5)) ** -0.5
)
df['NLI'] = df['02Blue'] ** 2 - df['04Red'] ** 2 + df['04Red']
df['Norm NIR'] = df['08NIR'] / (df['08NIR'] + df['03Green'] + df['04Red'])
df['Norm R'] = df['08NIR'] + df['04Red'] + df['03Green']
df['PPR'] = (df['05Veg5'] - df['04Red']) / (df['05Veg5'] + df['04Red'])
df['PVR'] = (df['02Blue'] - df['04Red']) / (df['02Blue'] + df['04Red'])
df['ND774/677'] = (df['06Veg6'] - df['05Veg5']) / (df['06Veg6'] + df['05Veg5'])
df['ND782/666'] = (df['07Veg7'] - df['06Veg6']) / (df['07Veg7'] + df['06Veg6'])
df['ND790/670'] = (df['05Veg5'] - df['03Green']) / (df['05Veg5'] + df['03Green'])
df['ND800/2170'] = (df['13SWIR2'] - df['04Red']) / (df['13SWIR2'] + df['04Red'])
# 61:75
df['PSNDc1'] = (df['08NIR'] - df['02Blue']) / (df['08NIR'] + df['02Blue'])
df['PSNDb1'] = (df['08NIR'] - df['01Coastal']) / (df['08NIR'] + df['01Coastal'])
df['PSNDa1'] = (df['08NIR']- df['02Blue'])/(df['08NIR']+df['02Blue'])
df['ND800/680'] = (df['08NIR'] - df['07Veg7']) / (df['08NIR'] + df['07Veg7'])
df['NDII'] = (df['12SWIR1'] - df['10WV']) / (df['12SWIR1'] + df['10WV'])
df['NDII2'] = (df['13SWIR2']-df['01Coastal'])/(df['13SWIR2']+df['01Coastal'])
df['ND833/658'] = (df['13SWIR2'] - df['12SWIR1']) / (df['13SWIR2'] + df['12SWIR1'])
df['SIWSI'] = (df['01Coastal'] - df['10WV']) / (df['01Coastal'] + df['10WV'])
df['ND895/675'] = (df['02Blue'] - df['03Green']) / (df['02Blue'] + df['03Green'])
df['NDVI_M'] = (df['13SWIR2'] - df['08NIR']) / (df['13SWIR2'] + df['08NIR'])
df['BNDVI'] = (df['08NIR'] - df['02Blue']) / (df['08NIR'] + df['02Blue'])
df['GNDVI'] = (df['08NIR'] - df['03Green']) / (df['08NIR'] + df['03Green'])
# 76:90
df['CDVI'] = (df['08NIR'] - df['04Red']) / (df['08NIR'] + df['04Red'])
df['NBR'] = (df['08NIR'] - df['12SWIR1']) / (df['08NIR'] + df['12SWIR1'])
df['RI'] = df['04Red'] - df['03Green'] + df['03Green']
df['NDVI690-710'] = df['08NIR'] / (df['08NIR'] + df['07Veg7'])
df['NDVIc'] = (df['08NIR'] - df['04Red']) / (df['08NIR'] + df['04Red']) * (1 - (df['12SWIR1'] - df['13SWIR2']) / (df['12SWIR1'] - df['13SWIR2'].min()) / (df['12SWIR1'] - df['13SWIR2'].min()))
df['PNDVI'] = (df['08NIR'] - (df['03Green'] + df['04Red'] + df['02Blue'])) / (df['08NIR'] + (df['03Green'] + df['04Red'] + df['02Blue']))
df['PVI'] = (1/(df['08NIR']**2 + 1)) * (df['08NIR'] - df['04Red'] - df['02Blue'])
df['RARSa1'] = df['02Blue'] / df['03Green']
df['RARSa2'] = (df['06Veg6'] - df['02Blue']) / (df['06Veg6'] + df['02Blue'])
df['RARSa3'] = df['05Veg5'] / df['06Veg6']
df['RARSc3'] = df['08NIR'] / df['04Red'] * 800 / 500
df['RARSc4'] = df['08NIR'] * df['03Green'] / (df['08NIR'] + df['03Green'])
df['RDVI'] = ((df['08NIR'] - df['06Veg6']) / (df['08NIR'] + df['06Veg6']))**0.5
# 91:105
df['Rededge1'] = (df['05Veg5'] - df['04Red']) / (df['05Veg5'] + df['04Red'])
df['RBNDVI'] = (df['08NIR'] - (df['04Red'] + df['02Blue'])) / (df['08NIR'] + (df['04Red'] + df['02Blue']))
df['REIP1'] = 700 + 40 * ((df['06Veg6'] + df['07Veg7']) / 2 - (df['05Veg5'] * df['08NIR']) / (df['05Veg5'] - df['08NIR']))
df['REIP2'] = 702 + 40 * ((df['07Veg7'] + df['08NIR']) ** 2 - df['07Veg7'] * df['08NIR'] - df['07Veg7'])
df['REIP3'] = 705 + 35 * ((df['05Veg5'] + df['07Veg7'] ** 2) - df['05Veg5'] * df['06Veg6'] - df['05Veg5'])
df['REP'] = 700 + 40 * ((df['07Veg7'] + df['10WV'] * 2) - (df['01Coastal'] + df['08NIR'] * 2))
df['Rre'] = np.sqrt(df['01Coastal'] + (df['03Green']**2))
df['RDVI'] = (df['08NIR'] - df['05Veg5']) / (df['08NIR'] + df['05Veg5'])**(1/2)
df['SAVImir'] = (df['08NIR']-df['12SWIR1']) + (df['08NIR'] + df['12SWIR1'] + 1) ** 0.5
df['MSI2'] = df['12SWIR1'] / df['03Green']
df['SR16502218'] = df['08NIR'] / df['09NIRn']
df['SR440/740'] = df['02Blue'] / df['06Veg6']
df['BGI'] = (df['02Blue'] / df['03Green'])
# 106:120
df['SR550/670'] = df['03Green'] / df['06Veg6']
df['SR550/800'] = df['03Green'] / df['10WV']
df['GI'] = df['03Green'] / df['06Veg6']
df['560/658'] = df['03Green'] / df['04Red']
df['SR672/550'] = df['06Veg6'] / df['02Blue']
df['SR 672/708'] = df['06Veg6'] / df['07Veg7']
df['SR675/555'] = df['06Veg6'] / df['03Green']
df['SR675/700'] = df['06Veg6'] / df['07Veg7']
df['SR675/705'] = df['04Red'] / df['03Green']
df['SR700'] = df['07Veg7']
df['SR700_670'] = df['03Green'] / df['02Blue']
df['SR735/710'] = df['03Green'] / df['02Blue']
df['SR774/677'] = df['08NIR'] / df['03Green']
df['800/2170'] = df['02Blue'] / df['09NIRn']
# 121:135
df['PSSRc1'] = df['08NIR'] / df['03Green']
df['PSSRb1'] = df['08NIR'] / df['04Red']
df['RVI'] = df['03Green'] / df['02Blue']
df['PSSRa1'] = df['12SWIR1'] / df['07Veg7']
df['SR800/680'] = df['08NIR'] / df['06Veg6']
df['SR801/550'] = df['08NIR'] / df['03Green']
df['PBI'] = df['12SWIR1'] / df['05Veg5']
df['SR833_1649'] = df['03Green'] / df['10WV']
df['SR833/658'] = df['07Veg7'] / df['06Veg6']
df['Datt2'] = df['07Veg7'] / df['06Veg6']
df['SR860/550'] = df['08NIR'] / df['03Green']
df['RDI'] = df['09NIRn'] / df['08NIR']
df['SRMIR/Red'] = df['13SWIR2'] / df['04Red']
df['SRnir/700-715'] = df['08NIR']
# 136:150
df['SRNIR/MIR'] = df['08NIR'] / df['09NIRn']
df['RRI1'] = df['08NIR'] / df['04Red']
df['SRRedNIR'] = df['04Red'] / df['08NIR']
df['SRSWIRI/NIR'] = df['12SWIR1'] / df['08NIR']
df['SB885'] = df['06Veg6']
df['SAVI'] = ((df['08NIR'] - df['04Red']) * (1 + 0.5)) / (df['08NIR'] + df['04Red'] + 0.5)
df['SARVI'] = (1 + 0.5) * df['08NIR'] - (0.2 * (df['04Red'] - df['03Green'])) + (0.1 * (df['04Red'] - df['03Green'])) + (0.5)
df['SARVI2'] = 2.5*df['08NIR'] - df['04Red'] + df['08NIR'] + 6*df['04Red'] - 7.5*df['02Blue']
df['SAVI3'] = (1 + 0.5) * (df['08NIR'] - df['04Red']) / (df['08NIR'] + df['04Red'] + 0.5)
df['SAVI2'] = ((df['08NIR'] - df['04Red']) * (1 + 0.5)) / (df['08NIR'] + df['04Red'] + 0.5)
df['SLAVI'] = df['08NIR'] + df['04Red'] + df['13SWIR2']
df['SQRT_IR/R'] = np.sqrt(df['08NIR'] / df['04Red'])
df['SIPI1'] = (df['04Red'] - df['03Green']) / (df['04Red'] - df['06Veg6'])
df['SBI'] = 0.3037 * df['03Green'] + 0.2793 * df['04Red'] + 0.4743 * df['07Veg7'] + 0.5585 * df['09NIRn'] + 0.5082 * df['10WV'] + 0.1863 * df['06Veg6']
# 151:165
df['NSIMSS'] = (-0.016 * df['05Veg5'] + 0.131 * df['06Veg6'] - 0.425 * df['07Veg7'] + 0.882 * df['09NIRn'])
df['SBIMSS'] = 0.332 * df['03Green'] + 0.603 * df['04Red'] + 0.675 * df['07Veg7'] + 0.262 * df['09NIRn']
df['GVI'] = -0.2848 * df['01Coastal'] - 0.2435 * df['02Blue'] - 0.5436 * df['06Veg6'] + 0.7243 * df['08NIR'] + 0.0840 * df['09NIRn'] - 0.1800 * df['10WV']
df['YVIMSS'] = (-0.899 * df['07Veg7'] + 0.428 * df['10WV'] +
                0.076 * df['08NIR'] - 0.041 * df['09NIRn'])
df['TCARI/OSAVI'] = (3 * (df['08NIR'] - df['06Veg6']) - 0.2 * (df['08NIR'] - df['02Blue'])) / ((df['08NIR'] + df['06Veg6']) * (1 + 0.16) * (df['09NIRn'] - df['06Veg6']) / (df['09NIRn'] + df['06Veg6'] + 0.16))
df['TCARI'] = 3 * ((df['08NIR'] - df['07Veg7']) - 0.2 * (df['08NIR'] - df['03Green']) / (df['08NIR'] - df['07Veg7']))
df['TNDVI'] = (df['08NIR'] - df['04Red']) / (df['08NIR'] + df['04Red'] + 0.5) ** 0.5
df['TSAVI'] = (df['08NIR'] - df['04Red'] - df['03Green']) / (df['08NIR'] - df['03Green'] + 0.5 * (df['08NIR'] - df['04Red'] - df['03Green']) ** 2)
df['TSAVI'] = (0.5 * df['08NIR'] - 0.5 * df['04Red'] - 0.5 * df['04Red'] * (df['08NIR'] - 0.5 * df['04Red'])) / (df['08NIR'] + 0.5 * df['04Red'])
df['TVI'] = df['NDVI'] ** 0.5 + 0.5
df['TCI'] = 1.2 * (df['03Green'] - df['02Blue']) - 1.5 * (df['06Veg6'] - df['02Blue']) * ((df['03Green'] + df['06Veg6'])/2)**0.5
df['VI700'] = (df['03Green']*2 - df['02Blue'] - df['04Red'])
df['VARIgreen'] = (df['03Green'] - df['06Veg6']) / (df['03Green'] + df['06Veg6']) - df['01Coastal']
df['VARI700'] = ((df['03Green'] - (1.7 * df['07Veg7'])) + (0.7 * df['02Blue'])) / (df['03Green'] + (2.3 * df['07Veg7']) - (1.3 * df['02Blue']))
# 165:167
df['WDVI'] = df['08NIR'] - (df['04Red'] * 0.6)
df['WDRVI'] = ((df['08NIR'] - df['04Red']) / (df['08NIR'] + df['04Red'])) * 100

# Important_10_feature = ['id','12SWIR1','10WV','13SWIR2','09NIRn','01Coastal','05Veg5','CVI','NDRE','NBR','02Blue']
# df = df[Important_10_feature]
df